In [3]:
import tensorflow as tf
from keras import Sequential
from keras.models import load_model

from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, adam
from keras.utils import np_utils
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from numpy import random 
from PIL import Image
from keras.utils import np_utils
#import theano

In [18]:
path_test = "C:/Users/Daved/OneDrive/Desktop/Raven/Coffee/"

CATEGORIES = ["Empty", "Grade1", "Grade2", "Grade3", "Grade4"]
IMG_SIZE = 150

In [19]:
training = []

def createTrainingData():
  for category in CATEGORIES:
    path = os.path.join(path_test, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img))
      new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
      training.append([new_array, class_num])
      
createTrainingData()

In [20]:
random.shuffle(training)

In [45]:
X =[]
y =[] 

for features, label in training:
  X.append(features)
  y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [46]:
print(X.size)
print("Memory size of one array element in bytes: ",
      X.itemsize)
print("Memory size of numpy array in megabytes:",
      (X.size * X.itemsize) / 1000000 )

2195167500
Memory size of one array element in bytes:  1
Memory size of numpy array in megabytes: 2195.1675


In [47]:
X = X.astype('float32')
X /= 255

Y = np_utils.to_categorical(y, 5)
print(Y[100])
print(np.shape(Y))

[0. 0. 0. 0. 1.]
(32521, 5)


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

In [52]:
batch_size = 16
nb_classes = 5
nb_epochs = 20
img_rows, img_columns = 200, 200
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

In [59]:
cnn = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(5,  activation=tf.nn.softmax)
])

In [60]:
cnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [61]:
X_train = np.asarray(X_train)
X_test  = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test  = np.asarray(y_test)

In [62]:
cnv = cnn.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))


Epoch 1/20
1626/1626 [==============================] - 337s 206ms/step - loss: 0.3381 - accuracy: 0.8747 - val_loss: 0.0859 - val_accuracy: 0.9728
Epoch 2/20
1626/1626 [==============================] - 311s 191ms/step - loss: 0.0771 - accuracy: 0.9742 - val_loss: 0.0892 - val_accuracy: 0.9736
Epoch 3/20
1626/1626 [==============================] - 309s 190ms/step - loss: 0.0488 - accuracy: 0.9834 - val_loss: 0.0266 - val_accuracy: 0.9922
Epoch 4/20
1626/1626 [==============================] - ETA: 0s - loss: 0.0421 - accuracy: 0.9865

In [25]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 75, 75, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 37, 37, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 37, 37, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 43808)            

In [24]:
score = cnn.evaluate(X_test, y_test, verbose = 0 )
print("Test Score: ", score[0])
print("Test accuracy: ", score[1])

Test Score:  0.005650412291288376
Test accuracy:  0.9993447065353394


In [26]:
import matplotlib.pyplot as plt
plt.plot(cnn.history['loss'], label='train loss')
plt.plot(cnn.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(cnn.history['accuracy'], label='train acc')
plt.plot(cnn.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

TypeError: 'History' object is not subscriptable

In [ ]:
cnn.save("coffCNN.h5")

In [2]:
cnn = load_model("coffCNN.h5")

In [3]:
cnn.save("coffCNN.pb")

INFO:tensorflow:Assets written to: coffCNN.pb\assets


INFO:tensorflow:Assets written to: coffCNN.pb\assets
